# DNN

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.preprocessing import Normalizer

In [ ]:
np.random.seed(1337)

In [ ]:
DATA = './data/'
TEST = 0.3

In [4]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [5]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [6]:
class_to_idx={d:i for i,d in enumerate(traindata['Y'].unique())}
traindata['Y']=traindata['Y'].replace(class_to_idx)
testdata['Y']=testdata['Y'].replace(class_to_idx)

In [7]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [8]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [9]:
y_train=Y
y_test= C

In [10]:
model = Sequential()
model.add(Dense(10, activation="relu", input_shape=X_train.shape[1:]))
model.add(Dropout(0.1))
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(23, activation="softmax"))

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-27 13:12:18.347779: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-27 13:12:18.348005: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# define optimizer and objective, compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                420       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               1408      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 23)                2967      
                                                                 
Total params: 4,795
Trainable params: 4,795
Non-trainable params: 0
_________________________________________________________________


In [13]:
(X_train.shape, y_train.shape)

((3673823, 41), (3673823,))

In [14]:
# train
model.fit(X_train, y_train, epochs=1, validation_split=0.3, shuffle=True, batch_size=1024)

2022-04-27 13:12:19.178528: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-27 13:12:20.254395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2512/2512 [==============================] - ETA: 0s - loss: 0.1545 - accuracy: 0.9560

2022-04-27 13:12:52.293085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2512/2512 [==============================] - 39s 14ms/step - loss: 0.1545 - accuracy: 0.9560 - val_loss: 0.0139 - val_accuracy: 0.9946


In [16]:
model.evaluate(X_test, y_test, batch_size=1024)

1196/1196 [==============================] - 9s 7ms/step - loss: 1.0615 - accuracy: 0.8225


[1.0615369081497192, 0.8225268721580505]